In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
!pip install transformer_lens -q
!pip install --upgrade attrs -q
!pip uninstall jax jaxlib -y -q
!pip install jax jaxlib -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.9.2 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.2.0 which is incompatible.


In [2]:
from transformer_lens import HookedTransformer

import json
import time

from src.neuron_texts import find_neurons

In [3]:
# Parameters
# model_name = "pythia-410m"
model_name = "gpt2-large"
device = "cuda"
# layers = [9, 10, 11]
# layers = [20, 21, 22, 23]
layers = [31, 32, 33, 34, 35]
num_neurons_per_layer = 20
rejected_neurons = [()]

In [4]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    # refactor_factored_attn_matrices=True,
    device=device,
)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-large into HookedTransformer


In [17]:
import random
from fancy_einsum import einsum
import torch

def find_random_neurons(model, layers, num_neurons_per_layer, rejected_neurons):
    results = {}
    rejected_neurons_with_tokens = []

    # Handle layers if they are negative
    num_layers = model.cfg.n_layers
    layers = [layer if layer >= 0 else num_layers + layer for layer in layers]

    for layer_num in layers:
        n_layer_neurons = model.W_out[layer_num, :, :]
        unembedding = model.W_U
        dot_product = einsum("neuron embed, embed token -> neuron token", n_layer_neurons, unembedding)
        
        values, indices = torch.max(dot_product, dim=-1) # Get the highest congruence with any given token for each neuron
        top_values, top_indices = torch.topk(values, indices.shape[0])


        found_i = []
        while len(found_i) < num_neurons_per_layer:
            i = random.randint(0, len(top_indices))
            if i in found_i:
                continue
            else:
                str_token = model.to_string(indices[top_indices][i])
                found_i.append(i)
                neuron_index = top_indices[i].item()
                results[(layer_num, neuron_index)] = {'token': str_token, 'congruence': top_values[i].item()}

    return results, None

In [9]:
# rejected_neurons = [(9, 2322), (9, 2894), (11, 3033)] # GPT-2 Small, 20 neurons per layer
# rejected_neurons = [(23, 3671), (23, 3177)] # GPT-2 Medium, 20 neurons per layer
# rejected_neurons = [(31, 4172), (31, 4899), (32, 4361), (33, 1582), (33, 122), (33, 2587), 
#                     (34, 2978), (34, 805), (35, 274), (35, 920), (35, 4849), (35, 684), 
#                     (35, 4396), (35, 295), (35, 3065), (35, 989)]  # GPT-2 Large, 40 neurons per layer
# rejected_neurons = [(44, 1657), (45, 284),(45, 4415) ,(46, 1785),(46, 1795),(47, 2065),
                    # (47, 1147),(47, 3489), (47, 4845),(47, 5429),(47, 4019),
                    # (43, 403),(47, 6356),(47, 3775),(47, 3635),] # GPT-2 XL, 20 neurons per layer

# rejected_neurons = [(16, 2727), (16, 523), (17, 4787),(18, 2078),(18, 123),(18, 4681),(19, 5095),(19, 4034),(20, 2259),(16, 4244),(20,795)] # GPT-2-large mid

# rejected_neurons = [(9, 2871), (10, 741), (10, 336), (11, 1266) , (11, 493) ,(11, 817) ,(11, 922),(11, 2790),(11, 1953) ,(11, 2209) ,(11, 2631) ,(11, 2640) ,(11, 803) ,(11, 875) ,(11, 2856),(11, 1240) ,(11, 339),(11, 1248),(11, 1994),
                    # (11, 1828),(11, 1076),(11, 1347), (11,110), (11,45), (11, 334), (11,1932)] # pythia 160m
# rejected_neurons = [(10, 457), (11,2627)]


# Update this until you are satisfied with the results

neuron_results, rejected_neurons_with_token = find_neurons(
    model=model,
    layers=layers,
    num_neurons_per_layer=num_neurons_per_layer,
    rejected_neurons=rejected_neurons,
)

for k, v in neuron_results.items():
    print(k, f"'{v['token']}'")

(20, 3772) ' that'
(20, 1625) ' of'
(20, 2216) ' and'
(20, 3388) ' at'
(20, 2152) ' that'
(20, 1477) ' right'
(20, 2446) ' over'
(20, 718) ' of'
(20, 1234) ' one'
(20, 1365) ' would'
(20, 2937) ' or'
(20, 1198) ' so'
(20, 2897) ' even'
(20, 3622) ' to'
(20, 2451) ' of'
(20, 1334) ' so'
(20, 144) ' the'
(20, 3711) ' to'
(20, 2950) ' had'
(20, 3809) ' was'
(21, 1863) ' about'
(21, 2711) ' or'
(21, 110) ' can'
(21, 3412) ' have'
(21, 3120) ' in'
(21, 1215) ' too'
(21, 1832) ' of'
(21, 758) ' Its'
(21, 2501) ' for'
(21, 4076) ' at'
(21, 3267) ' on'
(21, 354) ' for'
(21, 3247) ' time'
(21, 825) ' its'
(21, 808) ' in'
(21, 169) ' work'
(21, 3537) ' issue'
(21, 1725) ' would'
(21, 4050) ' that'
(21, 1017) ' no'
(22, 2896) ' one'
(22, 1074) ' not'
(22, 974) ' do'
(22, 3561) ' on'
(22, 855) ' the'
(22, 1166) ' current'
(22, 3822) ' and'
(22, 1268) ' by'
(22, 911) ' at'
(22, 1918) ' and'
(22, 1661) ' on'
(22, 2005) ' in'
(22, 3667) ' all'
(22, 2146) ' based'
(22, 1762) ' state'
(22, 961) ' that'

In [18]:
random_neuron_results, _ = find_random_neurons(
    model=model,
    layers=layers,
    num_neurons_per_layer=num_neurons_per_layer,
    rejected_neurons=rejected_neurons,
)

for k, v in random_neuron_results.items():
    print(k, f"'{v['token']}'")

(31, 396) ' Diver'
(31, 2420) ' Dragonbound'
(31, 160) ' arms'
(31, 2798) 'affe'
(31, 4364) '�'
(31, 302) 'safety'
(31, 3564) ' like'
(31, 1788) 'hus'
(31, 2959) ' Badge'
(31, 1055) 'ğ'
(31, 4457) ' Burk'
(31, 1790) 'etta'
(31, 2443) ' in'
(31, 4543) 'ructure'
(31, 4173) 'cel'
(31, 2459) ' Central'
(31, 751) 'ounces'
(31, 3811) 'eworthy'
(31, 365) 'ilst'
(31, 3352) ' guiActiveUnfocused'
(32, 2304) ' glimps'
(32, 1710) ' Dy'
(32, 2100) 'agy'
(32, 4063) 'ANS'
(32, 4569) ' also'
(32, 1993) ' Material'
(32, 3349) ' case'
(32, 4722) ' coverage'
(32, 3629) ' the'
(32, 2999) ' ©'
(32, 2285) 'sn'
(32, 3430) 'IME'
(32, 63) ' and'
(32, 4325) ' ba'
(32, 3669) ' Definition'
(32, 4563) 'iably'
(32, 2954) 'umo'
(32, 365) 'ern'
(32, 1449) ' right'
(32, 4973) ' free'
(33, 2844) ' ending'
(33, 3552) 'anski'
(33, 3193) ' waiver'
(33, 4388) 'esville'
(33, 2896) ' phot'
(33, 539) ' Kis'
(33, 386) ' sober'
(33, 2766) ' Gilbert'
(33, 4757) ' clad'
(33, 4695) 'track'
(33, 1546) 'acci'
(33, 2260) 'APD'
(33, 4

In [21]:
## Output of this notebook
output = {
    'parameters': {
        'model_name': model_name,
        'layers': layers,
        'num_neurons_per_layer': num_neurons_per_layer,
        'rejected_neurons': rejected_neurons_with_token,
    },
    'neurons': {str(k): v for k,v in neuron_results.items()},
}

# Save the output to a in ../data/neurons/
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
filename = f"{timestamp}_{model_name}.json"

with open(f"./experiment_data/1_next_token_neurons/{filename}", 'w') as f:
    json.dump(output, f, indent=4)

In [22]:
## Output of this notebook, for random neuron
output = {
    'parameters': {
        'model_name': model_name,
        'layers': layers,
        'num_neurons_per_layer': num_neurons_per_layer,
        # 'rejected_neurons': rejected_neurons_with_token,
        'rejected_neurons': [],
    },
    'neurons': {str(k): v for k,v in random_neuron_results.items()},
}

# Save the output to a in ../data/neurons/
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
filename = f"{timestamp}_{model_name}_random.json"

with open(f"./experiment_data/1_next_token_neurons/{filename}", 'w') as f:
    json.dump(output, f, indent=4)